# Evaluate
One of the most important steps in preparing Agents is setting a good evaluation framework. Foundation models change. Frameworks change. Data changes. Having a good set of benchmarks to evaluate against serves as both a regression test, and a benchmark upon which to improve through prompt engineering of your agents and tools, working on your data, and testing different foundation models.

In [0]:
%pip install databricks-agents databricks-langchain langgraph==0.3.4 
dbutils.library.restartPython()

In [0]:
import yaml
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)
catalog = config['catalog']
schema = config['schema']

In [0]:
from pyspark.sql import Row

data = [
    Row(
        request="What are the top 3 campaigns by opens?",
        expected_facts=[
            "Warm Hearts, Warm Plates (campaign_id: 120) with 1,071 opens",
            "Conquer the Kitchen (campaign_id: 149) with 997 opens",
            "Dive into Wealth (campaign_id: 137) with 909 opens"
        ],
        guidelines=[
            "The response must be concise and include three campaigns"
        ]
    ),
    Row(
        request="What is the cost of all the campaigns between January and February 2024?",
        expected_facts=[
            "Based on the data provided, the total cost of all campaigns between January and February 2024 is $17,383.80."
        ],
        guidelines=[
            "The response should say `based on the data provided`"
        ]
    )
]

spark.createDataFrame(data).write.mode("overwrite").saveAsTable(f"shm.marketing.agent_evals")

In [0]:
retrieval_records = spark.sql(
    "SELECT CAST(campaign_id AS STRING) AS doc_uri, campaign_description AS content FROM shm.marketing.campaigns_fixed"
)
display(retrieval_records)

In [0]:
import mlflow
from databricks.agents.evals import generate_evals_df
import pandas as pd
import math

agent_description = """
The Agent is a creative marketing generator that uses previous campaigns to generate new and novel campaigns. It is specifically designed to generate marketing campaigns that are both creative and tailored to cusomter personas.
"""

question_guidelines = """
# Example questions
- Generate a new premium campaign for Viking refrigerators emphasizing durability, advanced lighting, and air purification for customers upgrading their kitchen.
- Design a campaign for the Nest Protect 2nd Generation tailored to tech-savvy homeowners.

# Additional Guidelines
- The answer should be a generated campaign slogan
- The expected facts should be as concise as possible
"""

evals = generate_evals_df(
    retrieval_records,
    num_evals=10,
    agent_description=agent_description,
    question_guidelines=question_guidelines
)

In [0]:
display(evals)

In [0]:
spark.createDataFrame(evals).write.mode("overwrite").saveAsTable("shm.marketing.synthetic_evals")

In [0]:
%sql
SELECT slice(expected_facts, 1, 3) FROM shm.marketing.synthetic_evals

In [0]:
%sql
CREATE OR REPLACE TABLE shm.marketing.combined_evals
AS
SELECT 
    request.messages[0].content AS request, 
    slice(expected_facts, 1, 3) AS expected_facts, 
    array('') AS guidelines 
FROM 
    shm.marketing.synthetic_evals
UNION
SELECT 
    request, 
    expected_facts, 
    guidelines 
FROM 
    shm.marketing.agent_evals

Now we are going to evaluate the synthetic data set, as well as our SME developed data.

In [0]:
eval_set = spark.table("shm.marketing.combined_evals")

In [0]:
with mlflow.start_run(run_name="agent_eval") as run:
  mlflow.evaluate(
    data=eval_set,
    model='models:/shm.marketing.genie_agent/4',
    model_type="databricks-agent"
  )